# Using linear regression to predict the joints
***

### Datasets: [subset](http://cims.nyu.edu/~tompson/flic_plus.htm) of full [FLIC](http://vision.grasp.upenn.edu/cgi-bin/index.php?n=VideoLearning.FLIC)
- Number of all the training images is 17380
    - Divided into validation sample 17380 * 20% = 3476
    - Divided into training sample 17380 - (17380 * 20%) = 13904
- Number of test images is 3548
- Number of joints is 7 (x, y)
    
### Data preprocessing
- Using the ground truth coordinates of joints to decide a bounding box (human detector)
- Add some noise (padding, shift) to the bounding box by uniform distribution
- Crop the original image acoording to the bounding box
- Resize the cropped image to 100x100

### Algorithm: Linear regression, **y** = **XW** + **b**, and y is a vector coordinates of joints
- Model
    - Build a linear model for every coordinate (x, y) of joints, i.e., 7*2=14 linear model in total
    - Use L2-norm loss (square error) $\sum_i^n (f(x_i) - y_i)^2$ for each model
    - The total loss is the sum of that for every model
    - We can regard all the models as a single Neural Network (no hidden layer) model with 14 output for regression
- Optimization
    - Use Stochastic gradient descent (SGD) to update weights and biases with batch size 32
    - Use num_epochs=100 to repeat the training and optimization.
- Test
    - Compute the loss of validation data at the end of each epoch
    - Plot the loss decay based on each epoch
    - Plot original, transformed and predicted joints on the image

### Some implementation details
- Use CPU on Mac OS X 10.10.3

- Memeory analysis
>If using 220 x 220 size, the number of input features (raw pixels) will be `220*220*3`, which will take ~0.00108 GB in python. So with 17380 training sets, it will take about 17380 * 0.00108 ~= 18GB memory. And if we resize image to 110x110, it will take about ~4.5GB. 
      

In [1]:
# A bit of setup
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from IPython.core.display import clear_output
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Prepare for training

In [2]:
# set up fransform parameters to find the bounding box, crop and resize the images
# transfrom parameters
from my_scripts.transform import Transform
pad_inf = 1 # infimum scale
pad_sup = 1.5 # supremum scale
size = 100 # resize size
shift = 3 # radomly shift the bounding box
lcn = True # local contrast normalization
trans = Transform(pad_inf, pad_sup, size=size, shift=shift, lcn=lcn)

In [3]:
# set up the model and trainer for training
from my_scripts.regressor_trainer import NetworkTrainer
from models.LinearNet_flic import LinearNet_flic

data_dir = 'data/FLIC-full' # store image 
data_info_file = '%s/train_joints.csv' % data_dir # store corrdinates of joints for training data
model = LinearNet_flic(size)
trainer = NetworkTrainer(data_dir=data_dir, data_info_file=data_info_file,
                         model=model, trans=trans, sample_batches=False)

## Training

In [4]:
best_model, train_loss_history, val_loss_history = trainer.train_bgd()

100% (13904 of 13904) |###################| Elapsed Time: 0:01:42 Time: 0:01:42
 96% (48 of 50) |######################## | Elapsed Time: 0:03:25 ETA:  0:00:07

17380
Number of training data 13904
Number of validation data 3476
{'weight_decay': 0.0001, 'data_dir': 'data/FLIC-full', 'verbose': True, 'data_info_file': 'data/FLIC-full/train_joints.csv', 'sample_batches': False, 'learning_rate': 0.0001, 'update': 'SGD', 'batch_size': 32, 'reg': 0.0, 'self': <my_scripts.regressor_trainer.NetworkTrainer object at 0x10ddb1b10>, 'snapshot': 5, 'val_rate': 0.2, 'num_epochs': 100, 'gpu': -1, 'model': <models.LinearNet_flic.LinearNet_flic object at 0x10722ea50>, 'trans': <my_scripts.transform.Transform object at 0x1042bf250>, 'dropout': 1.0, 'momentum': 0, 'num_iters': 50}
epoch01	training mean loss =0.797548132653	 elapsed time=321.025662184 sec.

100% (50 of 50) |#########################| Elapsed Time: 0:03:29 Time: 0:03:29
 99% (3468 of 3476) |#################### | Elapsed Time: 0:00:27 ETA:  0:00:00


epoch01	validation mean loss =0.668794333338	 elapsed time=39.0921449661 sec.

100% (3476 of 3476) |#####################| Elapsed Time: 0:00:27 Time: 0:00:27
Saving 11.0 x 8.0 in image.
100% (13904 of 13904) |###################| Elapsed Time: 0:01:44 Time: 0:01:44
 96% (48 of 50) |######################## | Elapsed Time: 0:03:13 ETA:  0:00:08

...

100% (50 of 50) |#########################| Elapsed Time: 0:04:53 Time: 0:04:53
 99% (3475 of 3476) |#################### | Elapsed Time: 0:00:28 ETA:  0:00:00


epoch21	validation mean loss =0.310793039234	 elapsed time=40.5321040154 sec.

100% (3476 of 3476) |#####################| Elapsed Time: 0:00:28 Time: 0:00:28
Saving 11.0 x 8.0 in image.
 36% (5066 of 13904) |#######             | Elapsed Time: 0:00:45 ETA:  0:01:13

 36% (5079 of 13904) |#######             | Elapsed Time: 0:00:45 ETA:  0:01:12

KeyboardInterrupt: 